# Custom LLLM Agent

This notebook goes through how to create your own custom LLM agent.

An LLM agent consists of three parts:
    
    - Tools: The tools the agent has available to use.
    - LLMChain: The LLMChain that produces the text that is parsed in a certain way to determine which action to take.
    - OutputParser: This determines how to parse the LLMOutput into 
        
        
In this notebook we walk through how to create a custom LLM agent.

In [1]:
import sys
sys.path.append("../../../../../lang-chat/backend/")
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

In [2]:
from app.toolkits.open_api.toolkit import OpenAPIToolkit
urls = [
    "https://www.klarna.com/.well-known/ai-plugin.json"
]
def get_toolkits(urls):
    return list(map(OpenAPIToolkit.from_plugin_url, urls))
toolkits = get_toolkits(urls)
namespaces = "\n\n".join([tk.get_typescript_namespace() for tk in toolkits])
#print(namespaces)

In [3]:
from langchain.agents import Tool, AgentExecutor
from langchain.agents.agent import LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [4]:
template = """Answer the following questions as best you can. You have access to the following tools (but you do not need to use them):

{tools}

If you want use a tool you should write valid TypeScript code to call it. Please put all parameters and their values on their own line. If you use strings, they should be wrapped in double quotes not single. Any arguments to this function should be values that can be loaded as json.

For your response, you have two options:

If you do not need to use any tools, you can just respond directly to the user by using the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Final Answer: your response to user here

If you do need to use tools, you should use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: ```typescript
...
``` 
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

When repsonding in your final answer to the user, they have NO knowledge of any intermediate steps. So if there is any intermediate knowledge there you want them to know, you should make sure to return it as part of your final answer.

Remember, you do not need to use tools. For example, if the user asks a question that you KNOW the answer to, you can respond. If they ask a question that you do not know the answer to, but no tool can help, you should respond asking for clarification.

Previous conversation (the question could be a follow up to something here):

{chat_history}

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

In [5]:
shopify = toolkits[0]

In [6]:
class CustomPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        return template.format(**kwargs)

In [7]:
anthropic_template = """\n\nHuman: Answer the following questions as best you can. You have access to the following tools (but you do not need to use them):

{tools}

If you want use a tool you should write valid TypeScript code to call it. Please put all parameters and their values on their own line. If you use strings, they should be wrapped in double quotes not single. Any arguments to this function should be values that can be loaded as json.

For your response, you have two options:

If you do not need to use any tools, you can just respond directly to the user by using the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Final Answer: your response to user here

If you do need to use tools, you should use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: ```typescript
...
``` 
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

When repsonding in your final answer to the user, they have NO knowledge of any intermediate steps. So if there is any intermediate knowledge there you want them to know, you should make sure to return it as part of your final answer.

Remember, you do not need to use tools. For example, if the user asks a question that you KNOW the answer to, you can respond. If they ask a question that you do not know the answer to, but no tool can help, you should respond asking for clarification.

Previous conversation (the question could be a follow up to something here):

{chat_history}

Begin!

Question: {input}\n\nAssistant: Thought:{agent_scratchpad}"""
anthropic_template1 = """{chat_history}\n\nHuman: "You are a helpful assistant. A very smart language model.

You have access to the following tools. The tools are TypeScript functions.

{tools}

If you want use a tool you should write valid TypeScript code to call it, always saving the output to a const with name "langchain". You should only make one function call at a time. Any arguments to this function should be values that can be loaded as json - e.g. no infinity values, etc. Any fuction calls MUST abide by the typescript definitions listed above.

Return a helpful response to the user. If you need to call a tool to get more information, just write the code for the tool in between a [TOOL_USE_BEGIN] and [TOOL_USE_END] token markers. You will get a response from the tool in between a [TOOL_RESULT_BEGIN] and [TOOL_RESULT_END] tokens. 
The user you are are interacting with will not see the Tool invocation or the Tool repsonse. Therefor, if there is information there that answers their question, you must relay that information to the user.

An example interaction may look like:

<Begin Example>
Human: What's the weather today?
Assistant: [TOOL_USE_BEGIN]const langchain = weather.onDate({{query: "1/2/2021"}})[TOOL_USE_END][TOOL_RESULT_BEGIN]49 degrees[TOOL_RESULT_END]The weather today is 49 degrees.
<End Example>

Begin!

Question: {input}\n\nAssistant:{agent_scratchpad}"""
class CustomAnthropicPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"[TOOL_USE_END][TOOL_RESULT_BEGIN]{observation}[TOOL_RESULT_END]"
        kwargs["agent_scratchpad"] = thoughts
        return anthropic_template1.format(**kwargs)

In [8]:
anthropic_prompt = CustomAnthropicPromptTemplate(input_variables=["input", "intermediate_steps", "chat_history", "tools"])

In [9]:
prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps", "chat_history", "tools"])

In [10]:
PREFIX = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist."""


SUFFIX = """TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

{tools}

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me please, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool. Output should be in a typescript Markdown snippet.
If you want use a tool you should write valid TypeScript code to call it. Please put all parameters and their values on their own line. If you use strings, they should be wrapped in double quotes not single. Any arguments to this function should be values that can be loaded as json.


```typescript
...

```

**Option #2:**
Use this if you want to respond directly to the human. Normal text formatted in the following schema:

Final Answer:

USER'S INPUT
--------------------
Here is the user's input (remember to respond as instructed):

{input}"""

TEMPLATE_TOOL_RESPONSE = """TOOL RESPONSE: 
---------------------
{observation}

USER'S INPUT
--------------------

Okay, so what is the response to my last comment? If using information obtained from the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! Remember to respond as instructed originally."""
import json
from typing import Any, List, Optional, Sequence, Tuple

from langchain.agents.agent import Agent
from langchain.callbacks.base import BaseCallbackManager
from langchain.chains import LLMChain
from langchain.prompts.base import BasePromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    ChatPromptValue
)
from langchain.schema import (
    AgentAction,
    AIMessage,
    BaseLanguageModel,
    BaseMessage,
    BaseOutputParser,
    HumanMessage,
    SystemMessage,
    PromptValue,
)
from langchain.tools.base import BaseTool

class CustomChatPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs: Any) -> str:
        return self.format_prompt(**kwargs).to_string()
    
    def format_prompt(self, **kwargs: Any) -> PromptValue:
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        result = [SystemMessage(content=PREFIX)] + kwargs["chat_history"]
        intermediate_steps = kwargs.pop("intermediate_steps")
        result.append(HumanMessage(content=SUFFIX.format(**kwargs)))
        for action, observation in intermediate_steps:
            result.append(AIMessage(content=action.log))
            human_message = HumanMessage(
                content=TEMPLATE_TOOL_RESPONSE.format(observation=observation)
            )
            result.append(human_message)
        return ChatPromptValue(messages=result)

In [11]:
BASE_CHAT_TEMPLATE = CustomChatPromptTemplate(input_variables=["chat_history", "intermediate_steps", "input", "tools"])

In [12]:
FINAL_ANSWER_ACTION = "Final Answer:"
import json
import json5
from langchain.schema import AgentAction, AgentFinish
import re
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output):
        if FINAL_ANSWER_ACTION in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split(FINAL_ANSWER_ACTION)[-1].strip()},
                log=llm_output,
            )
        parsed = llm_output.split("Action:")[1].split("```typescript")[1].split("```")[0].strip()
        if parsed.startswith("const"):
            parsed = parsed.split(" = ")[1]
        typescript_str = parsed.split("(")[1].split(")")[0]
        tool_input = json.dumps(json5.loads(typescript_str))
        return AgentAction(tool=parsed.split("(")[0], tool_input=tool_input, log=llm_output)

In [13]:
output_parser = CustomOutputParser()

In [14]:
class CustomOutputParser1(AgentOutputParser):
    
    def parse(self, llm_output):
        if "[TOOL_USE_BEGIN]" not in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.strip()},
                log=llm_output,
            )
        parsed = llm_output.split("[TOOL_USE_BEGIN]")[1].strip()
        if parsed.startswith("const"):
            parsed = parsed.split(" = ")[1]
        typescript_str = parsed.split("(")[1].split(")")[0]
        tool_input = json.dumps(json5.loads(typescript_str))
        return AgentAction(tool=parsed.split("(")[0], tool_input=tool_input, log=llm_output)

## Test cases

In [15]:
from langchain.memory import ConversationBufferMemory
def get_agent(toolkits, llm):
    if isinstance(llm, OpenAI):
        prompt_to_use = prompt.partial(tools="\n\n".join([tk.get_typescript_namespace() for tk in toolkits]))
        memory = ConversationBufferMemory(memory_key="chat_history", )
        stop=["\nObservation:"]
        output_parser = CustomOutputParser()
    elif isinstance(llm, Anthropic):
        prompt_to_use = anthropic_prompt.partial(tools="\n\n".join([tk.get_typescript_namespace() for tk in toolkits]))
        memory = ConversationBufferMemory(memory_key="chat_history", )
        stop=["[TOOL_USE_END]"]
        output_parser = CustomOutputParser1()
    elif isinstance(llm, ChatOpenAI):
        prompt_to_use = BASE_CHAT_TEMPLATE.partial(tools="\n\n".join([tk.get_typescript_namespace() for tk in toolkits]))
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        stop=["\nObservation:"]
        output_parser = CustomOutputParser()
    else:
        raise ValueError
    prompt_to_use = anthropic_prompt.partial(tools="\n\n".join([tk.get_typescript_namespace() for tk in toolkits]))
    memory = ConversationBufferMemory(memory_key="chat_history", )
    stop=["[TOOL_USE_END]"]
    output_parser = CustomOutputParser1()
    llm_chain = LLMChain(llm=llm, prompt=prompt_to_use)
    agent = LLMSingleActionAgent(llm_chain=llm_chain, output_parser=output_parser, stop=stop)
    
    tools = []
    for tk in toolkits:
        tools.extend(tk.get_tools())
    return AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)


In [26]:
MEMORY_TEST_CASES = [
    # Should know the answer to "whats my name"
    [
        {"input": "hi im bob"},
        {"input": "whats my name?"}
    ],
    # Should not know the answer to "whats my name"
    [
        {"input": "whats my name?"}
    ],
]
KLARNA_TEST_CASES = [
    
    [
        {"input": "what are some blue shirts you have available?"},
    ],
    [
        {"input": "what is the most expensive t-shirt you have?"},
        {"input": "whats the price in dollars?"},
    ],
]
MILO_TEST_CASES = [
    [
        {"input": "whats magic today?"},
        {"input": "thanks!"}
    ]
]

In [27]:
TESTS = [
    {
        "urls":[
            "https://www.klarna.com/.well-known/ai-plugin.json"
        ],
        "test_cases": KLARNA_TEST_CASES
    }
#     {
#         "urls":[
#             "https://www.joinmilo.com/.well-known/ai-plugin.json"
#         ],
#         "test_cases": MILO_TEST_CASES
#     }
]

In [32]:
from langchain.llms import OpenAI, Anthropic
from langchain.chat_models import ChatOpenAI
llm = OpenAI(temperature=0, verbose=True)
#llm = ChatOpenAI(temperature=0)
#llm = Anthropic(temperature=0)

In [33]:
for test in TESTS:
    toolkits = get_toolkits(test["urls"])
    for tc in test["test_cases"]:
        agent = get_agent(toolkits, llm)
        for entry in tc:
            foo = agent(entry)



> Entering new AgentExecutor chain...
 [TOOL_USE_BEGIN]const langchain = KlarnaProducts.productsUsingGET({q: "blue shirt", size: 10, budget: 100})
[TOOL_USE_END]{'products': [{'name': 'Polo Ralph Lauren Polo Shirt - Blue', 'url': 'https://www.klarna.com/us/shopping/pl/cl10001/3200135914/Clothing/Polo-Ralph-Lauren-Polo-Shirt-Blue/?utm_source=openai', 'price': '$98.00', 'attributes': ['Material:Cotton', 'Target Group:Man', 'Color:Blue']}, {'name': 'Dickies Original Short Sleeve Work Shirt - Royal Blue', 'url': 'https://www.klarna.com/us/shopping/pl/cl10001/3200090586/Clothing/Dickies-Original-Short-Sleeve-Work-Shirt-Royal-Blue/?utm_source=openai', 'price': '$24.99', 'attributes': ['Material:Polyester,Cotton', 'Target Group:Man', 'Color:Blue']}, {'name': 'Hugo Boss Paddy Polo Shirt - Open Blue', 'url': 'https://www.klarna.com/us/shopping/pl/cl10001/3200882220/Clothing/Hugo-Boss-Paddy-Polo-Shirt-Open-Blue/?utm_source=openai', 'price': '$64.19', 'attributes': ['Material:Cotton', 'Target G

In [24]:
foo

{'input': 'what are some blue shirts you have available?',
 'chat_history': '',
 'output': 'Here are some blue shirt options from Klarna:\nPolo Ralph Lauren Polo Shirt - Blue\nDickies Original Short Sleeve Work Shirt - Royal Blue\nHugo Boss Paddy Polo Shirt - Open Blue\nLacoste Slim Fit Petit Pique Polo Shirt - Navy Blue\nCarhartt Long Sleeve T-shirt (available in gray, blue and green)'}

In [21]:
foo

{'input': 'what are some blue shirts you have available?',
 'chat_history': '',
 'output': 'I found 10 blue shirts that may interest you. Here are some of them: Polo Ralph Lauren Polo Shirt - Blue, Dickies Original Short Sleeve Work Shirt - Royal Blue, Hugo Boss Paddy Polo Shirt - Open Blue, Lacoste Slim Fit Petit Pique Polo Shirt - Navy Blue, Carhartt Long Sleeve T-shirt, Carhartt Flame-Resistant Force Long Sleeve T-Shirt - Blue, Psycho Bunny Mens Copa Gradient Logo Graphic Tee, La Martina Blue Shirt, Old Navy Softest Crew Neck T-shirt for Boys - Ink Blue (898709), La Martina Light Blue Shirt.'}